# MAKE MODEL DATA
## CLEAN AND RESTRUCTURE - PART4
Notebook to combine financial data, market data, and setup features which will be used by the model. We will be setting our target 'delisted' as 0 - company is still listed 1 - company delisted<br> 
Listed Companies - We take data from 2017
Delisted Companies - Companies that were delisted from 2015 onwards we look at their last available information. 

Read - restructure_identity.csv<br>
Read - cleaned_financial_statements.csv<br>
Read - cleaned_market_data.csv<br>
Write - model_data.csv

In [2]:
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns

# allow render in Jupyter notebook
%matplotlib inline
%pylab inline 
%config IPCompleter.greedy=True
%config InlineBackend.figure_format = 'svg'
sns.set_style("white")

Populating the interactive namespace from numpy and matplotlib


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### 1. READ CLEANED FINANCIAL DATA

In [4]:
# financial statement for companies
fin = pd.read_csv('../data/cleaned_financial_statements.csv')
fin = fin.drop(columns = 'Unnamed: 0')
fin['sa_finance1_year'] = pd.to_datetime(fin['sa_finance1_year'], format="%Y-%m-%d")
fin['sa_finance1_year']= fin['sa_finance1_year'].dt.year
fin.head()

/Users/user/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (65) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,sa_finance1_cocode,sa_company_name,sa_finance1_year,sa_total_income,sa_sales,sa_total_assets,sa_pat,sa_total_liabilities,sa_gross_working_capital_cosm,sa_current_assets_incl_st_invest_loans,sa_current_liabilities,sa_cash_bal,sa_st_trade_bills_receivables,sa_debtors_more_6m,sa_st_trade_receivables,sa_nf_operating_expenses,sa_f_operating_expenses,sa_dep_owned_ast,sa_retained_profits,sa_pbit,sa_interest_inc,sa_interest_exp,sa_st_inventories,sa_pbdita_net_of_peoifi,sa_pbpt_net_of_peoi,sa_total_capital,sa_gross_fixed_assets,sa_gross_fixed_assets.1,sa_pbdita,sa_cost_of_goods_sold,sa_cost_of_sales,sa_total_expense,sa_net_sales,sa_depreciation,sa_non_current_assets,sa_net_fixed_assets,sa_gross_fixed_assets.2,sa_gross_fixed_assets.3,sa_non_fin_services_inc,sa_inc_fin_serv,sa_net_working_capital,sa_non_current_liabilities,sa_tot_inc_net_of_pe,sa_sales_n_chg_in_stk,sa_chg_in_assets_with_no_chg_in_efficiency,sa_chg_in_efficiency_with_no_chg_in_assets,sa_chg_in_efficiency_on_chg_in_assets,sa_reported_pat,sa_diff_in_pat,sa_pbpt,sa_pbt,sa_pbit_net_of_peoifi,sa_cash_profit,sa_equity_div_inc_dist_pc_pat,sa_sales_net_fixed_assets,sa_total_income_avg_tot_asset_net_miscexp_now_reval,sa_chg_in_efficiency_in_use_of_assets,sa_pc_chg_in_efficiency_with_no_chg_in_assets,sa_pc_chg_in_assets_with_no_chg_in_efficiency,sa_pc_chg_in_efficiency_on_chg_in_assets,sa_total_div_pc_pat,sa_total_interest_exp,sa_nf_oper_exp_pc_total_exp,sa_f_oper_exp_pc_total_exp,sa_fin_serv_exp_pc_total_exp,sa_pbpt_net_of_peoi_inc_fin_serv,sa_pat_cont_ops_pc_inc_cont_ops,sa_pat_discont_ops_pc_inc_discont_ops,sa_total_provisions_pc_pbdita,sa_write_offs_pc_pbdita,sa_depreciation_pc_pbdita,sa_amortisation_pc_pbdita,sa_fin_serv_exp_pc_pbdita,sa_prov_direct_tax_pc_pbdita,sa_pat_pc_pbdita,sa_pbdita_pc_total_inc,sa_pbt_pc_total_inc,sa_pat_pc_total_inc,sa_cash_profit_pc_total_inc,sa_pat_net_of_pe_pc_tot_inc_net_of_pe,sa_pat_net_of_pe_pc_avg_networth,sa_pat_pc_avg_networth,sa_cash_profit_pc_avg_networth,sa_pbpt_net_of_peoi_pc_avg_capital_employed,sa_pat_net_of_pe_pc_avg_capital_employed,sa_pat_pc_avg_capital_employed,sa_pat_net_of_pe_pc_avg_tot_asset_net_menow_reval,sa_pc_chg_in_profitability_with_no_chg_in_income,sa_st_trade_bills_receivables_pc_current_assets_incl_st_invest_loans,sa_quick_ratio,sa_current_ratio,sa_debt_equity,sa_cash_n_bank_no_fd_security_current_liab_n_prov,sa_cash_n_bank_no_fd_security_cost_of_sales_per_day,sa_rawmat_stores_spares_avg_stk_rawmat_stores,sa_cost_of_goods_sold_avg_stk_fg,sa_sales_avg_debtors,sa_creditors_turnover,sa_total_inc_net_of_pe_compensation_to_empl,sa_sales_avg_gfa_net_of_reval,sa_sales_net_repairs_avg_nfa_net_of_reval
0,11,20 MICRONS LTD.,2014,3097.0,3010.4,3207.7,1.3,3207.7,1374.5154,1341.1,1605.4,1.5,539.0,15.1,539.0,2823.5,3040.4,100.8,1.3,221.3,4.5,202.7,578.4,238.1,-6.5,169.1,2038.7,2038.7,322.1,2022.22,2872.0,3146.9,2888.8,100.8,1866.6,1468.9,2038.7,2038.7,0.9,73.1,-391.5,757.1,3084.8,3061.6,357.5680,-144.6451,-17.8230,1.3,0.0,4.4,0.9,137.3,92.0,0.0000,204.9425,1.0300,0.5915,-74.1389,183.2742,-6.2238,0.0000,223.1,89.7232,96.6157,6.8925,-0.0889,0.0420,0,1.0866,0.0000,31.2946,0.0000,67.3393,-0.1242,0.4036,10.4004,0.0291,0.0420,2.9706,-0.2691,-1.1715,0.1835,12.9852,-0.2874,-0.3669,0.0575,-0.2760,97.6290,40.1909,0.4192,0.7688,2.1780,0.0389,8.3625,4.2690,12.7384,6.0298,5.1871,11.9104,1.5377,2.1030
1,11,20 MICRONS LTD.,2015,3316.2,3252.7,3289.8,-40.7,3289.8,1521.2025,1505.3,1484.2,2.1,442.7,20.8,442.7,3035.8,3285.1,94.5,-40.7,207.2,6.7,222.7,631.7,238.2,-65.9,169.1,2098.2,2098.2,301.7,2199.76,3108.5,3378.2,3116.7,94.5,1784.5,1428.0,2098.2,2098.2,0.9,39.7,-105.1,1001.6,3298.3,3274.0,249.2082,-27.7733,-2.2349,-40.7,0.0,-42.1,-46.1,143.7,41.7,NaN,227.7801,1.0208,0.9060,-12.6703,113.6899,2.1595,NaN,225.8,89.8644,97.2441,7.3797,-1.6599,-1.2273,0,1.2595,0.0663,31.3225,0.0000,82.6318,-1.7899,-13.4902,9.0978,-1.3901,-1.2273,1.2575,-1.7767,-8.1327,-5.6485,5.7872,-2.8595,-2.5427,-1.7660,-1.8038,92.4589,29.4094,0.4644,0.9333,2.2201,0.1059,19.5739,4.0029,1

In [5]:
fin.shape

(22135, 101)

### 2. READ CLEANED MARKET DATA

In [6]:
# market data for companies
mar = pd.read_csv('../data/cleaned_market_data.csv')
mar = mar.drop(columns = 'Unnamed: 0')
mar.head()

,co_code,co_stkdate,company_name,equity_facevalue_on_stkdate,eps_on_stkdate,equity_bv_on_stkdate,high_price,stock_returns,market_cap,pe,pb,traded_qty
0,11,2014,20 MICRONS LTD.,5.0,0.3194,756.4,42.00,1.0969,1274.8595,114.1287,1.7133,490250.0
1,11,2015,20 MICRONS LTD.,5.0,0.7793,816.7,47.60,1.2000,1548.7683,46.8348,2.1146,792926.0
2,11,2016,20 MICRONS LTD.,5.0,4.6477,943.1,47.00,1.1993,1524.3769,46.8990,1.6163,1738322.0
3,11,2017,20 MICRONS LTD.,5.0,4.4295,1250.3,63.95,1.1995,2214.2280,21.6022,1.7710,2699921.0
4,11,2018,20 MICRONS LTD.,5.0,5.5432,1382.7,68.00,1.1345,2244.2215,17.5467,1.7261,1151679.0


In [7]:
mar.shape

(20462, 12)

### 3. JOIN FINANCIAL AND MARKET DATA

In [8]:
### join financial and market data on company id and year

df = pd.merge(fin, mar,  how='left', left_on=['sa_finance1_cocode','sa_finance1_year'], right_on = ['co_code','co_stkdate'])


In [9]:
#drop extra columns after join
df= df.drop(columns=['co_code', 'co_stkdate','co_stkdate','company_name'])
df.head(20) 

,sa_finance1_cocode,sa_company_name,sa_finance1_year,sa_total_income,sa_sales,sa_total_assets,sa_pat,sa_total_liabilities,sa_gross_working_capital_cosm,sa_current_assets_incl_st_invest_loans,sa_current_liabilities,sa_cash_bal,sa_st_trade_bills_receivables,sa_debtors_more_6m,sa_st_trade_receivables,sa_nf_operating_expenses,sa_f_operating_expenses,sa_dep_owned_ast,sa_retained_profits,sa_pbit,sa_interest_inc,sa_interest_exp,sa_st_inventories,sa_pbdita_net_of_peoifi,sa_pbpt_net_of_peoi,sa_total_capital,sa_gross_fixed_assets,sa_gross_fixed_assets.1,sa_pbdita,sa_cost_of_goods_sold,sa_cost_of_sales,sa_total_expense,sa_net_sales,sa_depreciation,sa_non_current_assets,sa_net_fixed_assets,sa_gross_fixed_assets.2,sa_gross_fixed_assets.3,sa_non_fin_services_inc,sa_inc_fin_serv,sa_net_working_capital,sa_non_current_liabilities,sa_tot_inc_net_of_pe,sa_sales_n_chg_in_stk,sa_chg_in_assets_with_no_chg_in_efficiency,sa_chg_in_efficiency_with_no_chg_in_assets,sa_chg_in_efficiency_on_chg_in_assets,sa_reported_pat,sa_diff_in_pat,sa_pbpt,sa_pbt,sa_pbit_net_of_peoifi,sa_cash_profit,sa_equity_div_inc_dist_pc_pat,sa_sales_net_fixed_assets,sa_total_income_avg_tot_asset_net_miscexp_now_reval,sa_chg_in_efficiency_in_use_of_assets,sa_pc_chg_in_efficiency_with_no_chg_in_assets,sa_pc_chg_in_assets_with_no_chg_in_efficiency,sa_pc_chg_in_efficiency_on_chg_in_assets,sa_total_div_pc_pat,sa_total_interest_exp,sa_nf_oper_exp_pc_total_exp,sa_f_oper_exp_pc_total_exp,sa_fin_serv_exp_pc_total_exp,sa_pbpt_net_of_peoi_inc_fin_serv,sa_pat_cont_ops_pc_inc_cont_ops,sa_pat_discont_ops_pc_inc_discont_ops,sa_total_provisions_pc_pbdita,sa_write_offs_pc_pbdita,sa_depreciation_pc_pbdita,sa_amortisation_pc_pbdita,sa_fin_serv_exp_pc_pbdita,sa_prov_direct_tax_pc_pbdita,sa_pat_pc_pbdita,sa_pbdita_pc_total_inc,sa_pbt_pc_total_inc,sa_pat_pc_total_inc,sa_cash_profit_pc_total_inc,sa_pat_net_of_pe_pc_tot_inc_net_of_pe,sa_pat_net_of_pe_pc_avg_networth,sa_pat_pc_avg_networth,sa_cash_profit_pc_avg_networth,sa_pbpt_net_of_peoi_pc_avg_capital_employed,sa_pat_net_of_pe_pc_avg_capital_employed,sa_pat_pc_avg_capital_employed,sa_pat_net_of_pe_pc_avg_tot_asset_net_menow_reval,sa_pc_chg_in_profitability_with_no_chg_in_income,sa_st_trade_bills_receivables_pc_current_assets_incl_st_invest_loans,sa_quick_ratio,sa_current_ratio,sa_debt_equity,sa_cash_n_bank_no_fd_security_current_liab_n_prov,sa_cash_n_bank_no_fd_security_cost_of_sales_per_day,sa_rawmat_stores_spares_avg_stk_rawmat_stores,sa_cost_of_goods_sold_avg_stk_fg,sa_sales_avg_debtors,sa_creditors_turnover,sa_total_inc_net_of_pe_compensation_to_empl,sa_sales_avg_gfa_net_of_reval,sa_sales_net_repairs_avg_nfa_net_of_reval,equity_facevalue_on_stkdate,eps_on_stkdate,equity_bv_on_stkdate,high_price,stock_returns,market_cap,pe,pb,traded_qty
0,11,20 MICRONS LTD.,2014,3097.0,3010.4,3207.7,1.3,3207.7,1374.5154,1341.1,1605.4,1.5,539.0,15.1,539.0,2823.5,3040.4,100.8,1.3,221.3,4.5,202.7,578.4,238.1,-6.5,169.1,2038.7,2038.7,322.1,2022.22,2872.0,3146.9,2888.8,100.8,1866.6,1468.9,2038.7,2038.7,0.9,73.1,-391.5,757.1,3084.8,3061.6,357.5680,-144.6451,-17.8230,1.3,0.0,4.4,0.9,137.3,92.0,0.0000,204.9425,1.0300,0.5915,-74.1389,183.2742,-6.2238,0.0000,223.1,89.7232,96.6157,6.8925,-0.0889,0.0420,0,1.0866,0.0000,31.2946,0.0000,67.3393,-0.1242,0.4036,10.4004,0.0291,0.0420,2.9706,-0.2691,-1.1715,0.1835,12.9852,-0.2874,-0.3669,0.0575,-0.2760,97.6290,40.1909,0.4192,0.7688,2.1780,0.0389,8.3625,4.2690,12.7384,6.0298,5.1871,11.9104,1.5377,2.1030,5.0,0.3194,756.4,42.00,1.0969,1274.8595,114.1287,1.7133,490250.0
1,11,20 MICRONS LTD.,2015,3316.2,3252.7,3289.8,-40.7,3289.8,1521.2025,1505.3,1484.2,2.1,442.7,20.8,442.7,3035.8,3285.1,94.5,-40.7,207.2,6.7,222.7,631.7,238.2,-65.9,169.1,2098.2,2098.2,301.7,2199.76,3108.5,3378.2,3116.7,94.5,1784.5,1428.0,2098.2,2098.2,0.9,39.7,-105.1,1001.6,3298.3,3274.0,249.2082,-27.7733,-2.2349,-40.7,0.0,-42.1,-46.1,143.7,41.7,NaN,227.7801,1.0208,0.9060,-12.6703,113.6899,2.1595,NaN,225.8,89.8644,97.2441,7.3797,-1.6599,-1.2273,0,1.2595,0.0663,31.3225,0.0

In [10]:
df.shape

(22135, 110)

### 4. LOAD THE COMPANY IDENTITY DATA
Information on which company got delisted when or is still listed

In [11]:
# company identifier
cid = pd.read_csv('../data/restructure_identity.csv')
cid['delist_date'] = pd.to_datetime(cid['delist_date'], format="%Y-%m-%d")
cid = cid.drop(columns = 'Unnamed: 0')
cid.head()

,co_code,company_name,listed,delisted,delist_date
0,11,20 MICRONS LTD.,1,0,NaT
1,14,20TH CENTURY FINANCE CORPN. LTD. [MERGED],1,1,1999-05-28
2,289,A B C INDIA LTD.,1,0,NaT
3,301,A B C O PLASTICS LTD.,1,1,2004-07-02
4,363,STARLOG ENTERPRISES LTD.,1,1,2005-07-21


In [12]:
cid['delisted'].value_counts()

0    4657
1    3651
Name: delisted, dtype: int64

### 5. MAKE THE CRYSTAL BALL
Set aside data from 2018 to use in our predict the future api<br>
These are companies that have submitted their 2018 financials and are currenly listed

In [13]:
# take out data for 2018 companyies still listed . use this to predict the future in pur api
crystal_ball = df.copy() 

In [14]:
# still_listed - companies never been delisted before to predict the future aka whats going to happen in 2019??
still_listed = list(cid.loc[cid['delisted'] ==0,'co_code']) 
crystal_ball = crystal_ball.loc[crystal_ball['sa_finance1_cocode'].isin(still_listed) \
                                & (crystal_ball['sa_finance1_year'] >= 2018),:]
crystal_ball.shape

(1613, 110)

In [15]:
crystal_ball.isnull().sum()

sa_finance1_cocode                                                         0
sa_company_name                                                            0
sa_finance1_year                                                           0
sa_total_income                                                            6
sa_sales                                                                 231
sa_total_assets                                                            0
sa_pat                                                                     0
sa_total_liabilities                                                       0
sa_gross_working_capital_cosm                                              0
sa_current_assets_incl_st_invest_loans                                    42
sa_current_liabilities                                                    23
sa_cash_bal                                                              310
sa_st_trade_bills_receivables                                            224

In [16]:
crystal_ball.to_csv('../data/crystal_ball.csv')

### 6. MAKE MODEL DATA
First we select the listed companies and their data from 2017. Then pick up the delisted companies we pick up the last available information.

##### Extract data for companies that are listed in 2017

In [17]:
model_listed = df.copy()

In [18]:
# find companies not delist -- then get their 2017  fin statements
listed_2017 = list(cid.loc[cid['delisted'] ==0,'co_code']) 
len(listed_2017)

4657

In [19]:
# get their 2017 
model_listed = model_listed.loc[(model_listed['sa_finance1_year'] == 2017) &\
                                (model_listed['sa_finance1_cocode'].isin(listed_2017)),:]
model_listed['delisted']=False
model_listed.head()

,sa_finance1_cocode,sa_company_name,sa_finance1_year,sa_total_income,sa_sales,sa_total_assets,sa_pat,sa_total_liabilities,sa_gross_working_capital_cosm,sa_current_assets_incl_st_invest_loans,sa_current_liabilities,sa_cash_bal,sa_st_trade_bills_receivables,sa_debtors_more_6m,sa_st_trade_receivables,sa_nf_operating_expenses,sa_f_operating_expenses,sa_dep_owned_ast,sa_retained_profits,sa_pbit,sa_interest_inc,sa_interest_exp,sa_st_inventories,sa_pbdita_net_of_peoifi,sa_pbpt_net_of_peoi,sa_total_capital,sa_gross_fixed_assets,sa_gross_fixed_assets.1,sa_pbdita,sa_cost_of_goods_sold,sa_cost_of_sales,sa_total_expense,sa_net_sales,sa_depreciation,sa_non_current_assets,sa_net_fixed_assets,sa_gross_fixed_assets.2,sa_gross_fixed_assets.3,sa_non_fin_services_inc,sa_inc_fin_serv,sa_net_working_capital,sa_non_current_liabilities,sa_tot_inc_net_of_pe,sa_sales_n_chg_in_stk,sa_chg_in_assets_with_no_chg_in_efficiency,sa_chg_in_efficiency_with_no_chg_in_assets,sa_chg_in_efficiency_on_chg_in_assets,sa_reported_pat,sa_diff_in_pat,sa_pbpt,sa_pbt,sa_pbit_net_of_peoifi,sa_cash_profit,sa_equity_div_inc_dist_pc_pat,sa_sales_net_fixed_assets,sa_total_income_avg_tot_asset_net_miscexp_now_reval,sa_chg_in_efficiency_in_use_of_assets,sa_pc_chg_in_efficiency_with_no_chg_in_assets,sa_pc_chg_in_assets_with_no_chg_in_efficiency,sa_pc_chg_in_efficiency_on_chg_in_assets,sa_total_div_pc_pat,sa_total_interest_exp,sa_nf_oper_exp_pc_total_exp,sa_f_oper_exp_pc_total_exp,sa_fin_serv_exp_pc_total_exp,sa_pbpt_net_of_peoi_inc_fin_serv,sa_pat_cont_ops_pc_inc_cont_ops,sa_pat_discont_ops_pc_inc_discont_ops,sa_total_provisions_pc_pbdita,sa_write_offs_pc_pbdita,sa_depreciation_pc_pbdita,sa_amortisation_pc_pbdita,sa_fin_serv_exp_pc_pbdita,sa_prov_direct_tax_pc_pbdita,sa_pat_pc_pbdita,sa_pbdita_pc_total_inc,sa_pbt_pc_total_inc,sa_pat_pc_total_inc,sa_cash_profit_pc_total_inc,sa_pat_net_of_pe_pc_tot_inc_net_of_pe,sa_pat_net_of_pe_pc_avg_networth,sa_pat_pc_avg_networth,sa_cash_profit_pc_avg_networth,sa_pbpt_net_of_peoi_pc_avg_capital_employed,sa_pat_net_of_pe_pc_avg_capital_employed,sa_pat_pc_avg_capital_employed,sa_pat_net_of_pe_pc_avg_tot_asset_net_menow_reval,sa_pc_chg_in_profitability_with_no_chg_in_income,sa_st_trade_bills_receivables_pc_current_assets_incl_st_invest_loans,sa_quick_ratio,sa_current_ratio,sa_debt_equity,sa_cash_n_bank_no_fd_security_current_liab_n_prov,sa_cash_n_bank_no_fd_security_cost_of_sales_per_day,sa_rawmat_stores_spares_avg_stk_rawmat_stores,sa_cost_of_goods_sold_avg_stk_fg,sa_sales_avg_debtors,sa_creditors_turnover,sa_total_inc_net_of_pe_compensation_to_empl,sa_sales_avg_gfa_net_of_reval,sa_sales_net_repairs_avg_nfa_net_of_reval,equity_facevalue_on_stkdate,eps_on_stkdate,equity_bv_on_stkdate,high_price,stock_returns,market_cap,pe,pb,traded_qty,delisted
3,11,20 MICRONS LTD.,2017,3741.8,3729.3,3661.4,131.6,3661.4,1606.7038,1557.9,1399.7,1.6,548.6,NaN,548.6,3177.5,3405.7,89.8,131.6,429.8,5.6,216.8,591.7,509.0,196.6,176.4,2384.5,2384.5,519.6,2360.19,3319.6,3567.4,3569.9,89.8,2103.5,1560.3,2384.5,2384.5,18.5,5.6,-40.0,901.1,3738.3,3686.5,198.5575,14.8090,0.8336,131.6,0.0,201.6,193.2,419.2,247.7,0.0000,239.0117,1.0683,1.1476,6.9136,92.6972,1.2432,0.0000,216.8,89.0705,95.4673,6.3968,35.1071,3.5170,0,1.5012,0.1155,17.2825,0.0000,43.9184,11.8553,25.3272,13.8864,5.1633,3.5170,6.6198,3.4775,12.9039,13.0627,24.5868,7.8308,5.1780,5.2418,3.7114,75.0796,35.2141,0.5170,0.9745,1.2375,0.0499,10.7094,3.9417,12.2575,6.7237,4.0576,11.3042,1.6411,2.5176,5.0,4.4295,1250.3,63.95,1.1995,2214.2280,21.6022,1.7710,2699921.0,False
8,289,A B C INDIA LTD.,2017,1330.2,1314.2,784.4,-164.6,784.4,427.8439,531.2,363.0,7.1,356.7,154.4,356.7,1316.5,1363.3,27.4,-164.6,-12.1,3.8,43.5,6.7,-0.7,-71.0,54.1,624.0,624.0,15.3,1204.75,1344.2,1496.4,1282.6,27.4,253.2,222.8,624.0,624.0,1194.9,3.9,85.3,143.5,1319.8,1315.8,-205.8565,151.4126,-22.1561,-165.2,0.6,-58.9,-177.5,-28.1,-32.1,NaN,589.8564,1.5181,0.5100,-197.6666,268.7421,42.7933,NaN,43.5,87.9778,91.1053,3.1275,-18.2051,-12.3741,0,0.0000,775.1634

In [20]:
model_listed.shape

(4295, 111)

In [21]:
#model_listed.isnull().sum()

##### Extract companies that were delisted
We extract the last available information - financial and market for these companies

In [22]:
model_delisted = df.copy()

In [23]:
# list of companies that were delisted since 2015
delisted_companies = cid.loc[(cid['delisted']==1) & (cid['delist_date'].dt.year>=2015),'co_code']

In [23]:
model_delisted= model_delisted.loc[(model_delisted['sa_finance1_year']>=2014) & \
                                    (model_delisted['sa_finance1_year']<=2018) & \
                                    (model_delisted['sa_finance1_cocode'].isin(delisted_companies)),:]

In [24]:
model_delisted.shape

(1169, 108)

In [25]:
model_delisted = model_delisted\
.sort_values('sa_finance1_year', ascending=False)\
.drop_duplicates('sa_finance1_cocode').sort_index()

In [26]:
model_delisted['delisted']=True
model_delisted.shape

(475, 109)

In [27]:
model_delisted.head()

,sa_finance1_cocode,sa_company_name,sa_finance1_year,sa_total_income,sa_sales,sa_total_assets,sa_pat,sa_total_liabilities,sa_gross_working_capital_cosm,sa_current_assets_incl_st_invest_loans,sa_current_liabilities,sa_cash_bal,sa_st_trade_bills_receivables,sa_debtors_more_6m,sa_st_trade_receivables,sa_nf_operating_expenses,sa_f_operating_expenses,sa_dep_owned_ast,sa_retained_profits,sa_pbit,sa_interest_inc,sa_interest_exp,sa_st_inventories,sa_pbdita_net_of_peoifi,sa_pbpt_net_of_peoi,sa_total_capital,sa_gross_fixed_assets,sa_pbdita,sa_cost_of_goods_sold,sa_cost_of_sales,sa_total_expense,sa_net_sales,sa_depreciation,sa_non_current_assets,sa_net_fixed_assets,sa_gross_fixed_assets.1,sa_non_fin_services_inc,sa_inc_fin_serv,sa_net_working_capital,sa_non_current_liabilities,sa_tot_inc_net_of_pe,sa_sales_n_chg_in_stk,sa_chg_in_assets_with_no_chg_in_efficiency,sa_chg_in_efficiency_with_no_chg_in_assets,sa_chg_in_efficiency_on_chg_in_assets,sa_reported_pat,sa_diff_in_pat,sa_pbpt,sa_pbt,sa_pbit_net_of_peoifi,sa_cash_profit,sa_equity_div_inc_dist_pc_pat,sa_sales_net_fixed_assets,sa_total_income_avg_tot_asset_net_miscexp_now_reval,sa_chg_in_efficiency_in_use_of_assets,sa_pc_chg_in_efficiency_with_no_chg_in_assets,sa_pc_chg_in_assets_with_no_chg_in_efficiency,sa_pc_chg_in_efficiency_on_chg_in_assets,sa_total_div_pc_pat,sa_total_interest_exp,sa_nf_oper_exp_pc_total_exp,sa_f_oper_exp_pc_total_exp,sa_fin_serv_exp_pc_total_exp,sa_pbpt_net_of_peoi_inc_fin_serv,sa_pat_cont_ops_pc_inc_cont_ops,sa_pat_discont_ops_pc_inc_discont_ops,sa_total_provisions_pc_pbdita,sa_write_offs_pc_pbdita,sa_depreciation_pc_pbdita,sa_amortisation_pc_pbdita,sa_fin_serv_exp_pc_pbdita,sa_prov_direct_tax_pc_pbdita,sa_pat_pc_pbdita,sa_pbdita_pc_total_inc,sa_pbt_pc_total_inc,sa_pat_pc_total_inc,sa_cash_profit_pc_total_inc,sa_pat_net_of_pe_pc_tot_inc_net_of_pe,sa_pat_net_of_pe_pc_avg_networth,sa_pat_pc_avg_networth,sa_cash_profit_pc_avg_networth,sa_pbpt_net_of_peoi_pc_avg_capital_employed,sa_pat_net_of_pe_pc_avg_capital_employed,sa_pat_pc_avg_capital_employed,sa_pat_net_of_pe_pc_avg_tot_asset_net_menow_reval,sa_pc_chg_in_profitability_with_no_chg_in_income,sa_st_trade_bills_receivables_pc_current_assets_incl_st_invest_loans,sa_quick_ratio,sa_current_ratio,sa_debt_equity,sa_cash_n_bank_no_fd_security_current_liab_n_prov,sa_cash_n_bank_no_fd_security_cost_of_sales_per_day,sa_rawmat_stores_spares_avg_stk_rawmat_stores,sa_cost_of_goods_sold_avg_stk_fg,sa_sales_avg_debtors,sa_creditors_turnover,sa_total_inc_net_of_pe_compensation_to_empl,sa_sales_avg_gfa_net_of_reval,sa_sales_net_repairs_avg_nfa_net_of_reval,equity_facevalue_on_stkdate,eps_on_stkdate,equity_bv_on_stkdate,high_price,stock_returns,market_cap,pe,pb,traded_qty,delisted
17,400,A B L BIOTECHNOLOGIES LTD.,2016,0.1,0.1,443.5,-1.4,443.5,16.5000,83.4,141.4,NaN,NaN,NaN,NaN,0.3,0.3,0.7,-1.4,-1.4,NaN,NaN,NaN,-0.2,-1.4,82.0,345.7,-0.2,0.75,1.0,1.5,0.1,0.7,360.1,345.7,345.7,0.1,NaN,-59.6,122.4,0.1,0.1,NaN,NaN,NaN,-1.4,0.0,-1.4,-1.4,-1.4,-0.2,NaN,0.0289,0.0002,NaN,NaN,NaN,NaN,NaN,NaN,20.0000,20.0000,0.0000,NaN,-1400.0000,0.0,0.0000,0.0000,-350.0000,-250.0,0.0000,0.0000,700.0000,-200.0000,-1400.0000,-1400.0000,-200.0000,-1400.0000,-0.9933,-0.9933,-0.1419,-0.4513,-0.4513,-0.4513,-0.3446,100.0000,0.0000,0.0099,0.5785,1.1779,0.0099,511.0000,NaN,NaN,0.0606,0.0000,1.0000,0.0003,0.0003,10.0,-0.1585,142.2,0.96,1.0395,7.8720,NaN,0.0554,40210.0,True
89,2384,SCHNEIDER ELECTRIC PRESIDENT SYSTEMS LTD.,2017,2915.7,2908.1,1650.5,137.1,1650.5,928.6825,1193.1,847.6,NaN,742.7,74.8,742.7,2634.6,2660.5,52.1,137.1,257.1,0.8,22.1,318.5,304.4,231.4,60.5,836.0,309.2,2199.84,2655.3,2809.5,2705.7,52.1,457.4,358.8,836.0,162.0,5.0,204.7,223.0,2914.5,2939.0,226.7284,473.5931,49.5785,137.1,0.0,231.2,177.6,252.3,231.8,0.000,810.5072,1.9631,5.3279,63.1542,30.2345,6.7342,0.000,22.1,93.7747,94.6966,0.9219,46.2800,4.7021,0.0,16.0414,1.2937,16.8499,0.0,8.3765,13.0983,44.3402,10.6047,6.0912,4.7021,7.9501,4.7590,31.3907,31.0286,52.4612,32.8460,19.6877,19.4606,9.3385,5

##### Combine the data for delisted and listed companies 

In [28]:
# combine model_delisted, model_listed into one model_data
model_data = pd.concat([model_listed, model_delisted], ignore_index=True)
model_data.shape

(4770, 109)

In [29]:
model_data['sa_finance1_cocode'].nunique()

4770

In [30]:
model_data.head()

,sa_finance1_cocode,sa_company_name,sa_finance1_year,sa_total_income,sa_sales,sa_total_assets,sa_pat,sa_total_liabilities,sa_gross_working_capital_cosm,sa_current_assets_incl_st_invest_loans,sa_current_liabilities,sa_cash_bal,sa_st_trade_bills_receivables,sa_debtors_more_6m,sa_st_trade_receivables,sa_nf_operating_expenses,sa_f_operating_expenses,sa_dep_owned_ast,sa_retained_profits,sa_pbit,sa_interest_inc,sa_interest_exp,sa_st_inventories,sa_pbdita_net_of_peoifi,sa_pbpt_net_of_peoi,sa_total_capital,sa_gross_fixed_assets,sa_pbdita,sa_cost_of_goods_sold,sa_cost_of_sales,sa_total_expense,sa_net_sales,sa_depreciation,sa_non_current_assets,sa_net_fixed_assets,sa_gross_fixed_assets.1,sa_non_fin_services_inc,sa_inc_fin_serv,sa_net_working_capital,sa_non_current_liabilities,sa_tot_inc_net_of_pe,sa_sales_n_chg_in_stk,sa_chg_in_assets_with_no_chg_in_efficiency,sa_chg_in_efficiency_with_no_chg_in_assets,sa_chg_in_efficiency_on_chg_in_assets,sa_reported_pat,sa_diff_in_pat,sa_pbpt,sa_pbt,sa_pbit_net_of_peoifi,sa_cash_profit,sa_equity_div_inc_dist_pc_pat,sa_sales_net_fixed_assets,sa_total_income_avg_tot_asset_net_miscexp_now_reval,sa_chg_in_efficiency_in_use_of_assets,sa_pc_chg_in_efficiency_with_no_chg_in_assets,sa_pc_chg_in_assets_with_no_chg_in_efficiency,sa_pc_chg_in_efficiency_on_chg_in_assets,sa_total_div_pc_pat,sa_total_interest_exp,sa_nf_oper_exp_pc_total_exp,sa_f_oper_exp_pc_total_exp,sa_fin_serv_exp_pc_total_exp,sa_pbpt_net_of_peoi_inc_fin_serv,sa_pat_cont_ops_pc_inc_cont_ops,sa_pat_discont_ops_pc_inc_discont_ops,sa_total_provisions_pc_pbdita,sa_write_offs_pc_pbdita,sa_depreciation_pc_pbdita,sa_amortisation_pc_pbdita,sa_fin_serv_exp_pc_pbdita,sa_prov_direct_tax_pc_pbdita,sa_pat_pc_pbdita,sa_pbdita_pc_total_inc,sa_pbt_pc_total_inc,sa_pat_pc_total_inc,sa_cash_profit_pc_total_inc,sa_pat_net_of_pe_pc_tot_inc_net_of_pe,sa_pat_net_of_pe_pc_avg_networth,sa_pat_pc_avg_networth,sa_cash_profit_pc_avg_networth,sa_pbpt_net_of_peoi_pc_avg_capital_employed,sa_pat_net_of_pe_pc_avg_capital_employed,sa_pat_pc_avg_capital_employed,sa_pat_net_of_pe_pc_avg_tot_asset_net_menow_reval,sa_pc_chg_in_profitability_with_no_chg_in_income,sa_st_trade_bills_receivables_pc_current_assets_incl_st_invest_loans,sa_quick_ratio,sa_current_ratio,sa_debt_equity,sa_cash_n_bank_no_fd_security_current_liab_n_prov,sa_cash_n_bank_no_fd_security_cost_of_sales_per_day,sa_rawmat_stores_spares_avg_stk_rawmat_stores,sa_cost_of_goods_sold_avg_stk_fg,sa_sales_avg_debtors,sa_creditors_turnover,sa_total_inc_net_of_pe_compensation_to_empl,sa_sales_avg_gfa_net_of_reval,sa_sales_net_repairs_avg_nfa_net_of_reval,equity_facevalue_on_stkdate,eps_on_stkdate,equity_bv_on_stkdate,high_price,stock_returns,market_cap,pe,pb,traded_qty,delisted
0,11,20 MICRONS LTD.,2017,3741.8,3729.3,3661.4,131.6,3661.4,1606.7038,1557.9,1399.7,1.6,548.6,NaN,548.6,3177.5,3405.7,89.8,131.6,429.8,5.6,216.8,591.7,509.0,196.6,176.4,2384.5,519.6,2360.19,3319.6,3567.4,3569.9,89.8,2103.5,1560.3,2384.5,18.5,5.6,-40.0,901.1,3738.3,3686.5,198.5575,14.8090,0.8336,131.6,0.0,201.6,193.2,419.2,247.7,0.0000,239.0117,1.0683,1.1476,6.9136,92.6972,1.2432,0.0000,216.8,89.0705,95.4673,6.3968,35.1071,3.5170,0.0,1.5012,0.1155,17.2825,0.0,43.9184,11.8553,25.3272,13.8864,5.1633,3.5170,6.6198,3.4775,12.9039,13.0627,24.5868,7.8308,5.1780,5.2418,3.7114,75.0796,35.2141,0.5170,0.9745,1.2375,0.0499,10.7094,3.9417,12.2575,6.7237,4.0576,11.3042,1.6411,2.5176,5.0,4.4295,1250.3,63.95,1.1995,2214.2280,21.6022,1.7710,2699921.0,False
1,289,A B C INDIA LTD.,2017,1330.2,1314.2,784.4,-164.6,784.4,427.8439,531.2,363.0,7.1,356.7,154.4,356.7,1316.5,1363.3,27.4,-164.6,-12.1,3.8,43.5,6.7,-0.7,-71.0,54.1,624.0,15.3,1204.75,1344.2,1496.4,1282.6,27.4,253.2,222.8,624.0,1194.9,3.9,85.3,143.5,1319.8,1315.8,-205.8565,151.4126,-22.1561,-165.2,0.6,-58.9,-177.5,-28.1,-32.1,NaN,589.8564,1.5181,0.5100,-197.6666,268.7421,42.7933,NaN,43.5,87.9778,91.1053,3.1275,-18.2051,-12.3741,0.0,0.0000,775.1634,179.0850,0.0,305.8824,-84.3137,-1075.8170,1.1502,-13.3439,-12.3741,-2.41

### 7. WRITE TO FILE

In [31]:
# write to file -- for trace back to company names .. codes ... these will all be droped before modelling
model_data.to_csv('../data/model_data_india_with_codes.csv')

In [32]:
#  drop unwanted columns and make model ready
rm_headers= ['sa_finance1_cocode','sa_company_name','sa_finance1_year']
model_data = model_data.drop(columns = rm_headers)

In [33]:
#write data and pick up for modelling 
model_data.to_csv('../data/model_data_india.csv')

### next -> model.csv